Michał, Z., William, C., Karl, P., Oier, M., Chelsea, F., & Sergey, L. (2024). [Robotic Control via Embodied Chain-of-Thought Reasoning](https://arxiv.org/abs/2407.08693). arXiv preprint arXiv:2407.08693.

**Abstract**
<!-- A key limitation of learned robot control policies is their inability to generalize outside their training data. Recent works on vision-language-action models (VLAs) have shown that the use of large, internet pre-trained vision-language models as the backbone of learned robot policies can substantially improve their robustness and generalization ability. Yet, one of the most exciting capabilities of large vision-language models in other domains is their ability to reason iteratively through complex problems. Can that same capability be brought into robotics to allow policies to improve performance by reasoning about a given task before acting? Naive use of "chain-of-thought" (CoT) style prompting is significantly less effective with standard VLAs because of the relatively simple training examples that are available to them. Additionally, purely semantic reasoning about sub-tasks, as is common in regular CoT, is insufficient for robot policies that need to ground their reasoning in sensory observations and the robot state. To this end, we introduce Embodied Chain-of-Thought Reasoning (ECoT) for VLAs, in which we train VLAs to perform multiple steps of reasoning about plans, sub-tasks, motions, and visually grounded features like object bounding boxes and end effector positions, before predicting the robot action. We design a scalable pipeline for generating synthetic training data for ECoT on large robot datasets. We demonstrate, that ECoT increases the absolute success rate of OpenVLA, the current strongest open-source VLA policy, by 28% across challenging generalization tasks, without any additional robot training data. Additionally, ECoT makes it easier for humans to interpret a policy's failures and correct its behavior using natural language. -->
学习机器人控制策略的一个关键限制是它们无法在训练数据之外进行泛化. 最近关于视觉-语言-动作模型 (VLAs) 的研究表明, 使用大型、互联网预训练的视觉语言模型作为学习机器人策略的 backbone 可以大幅提高其鲁棒性和泛化能力. 然而, 大型视觉语言模型在其他领域最令人兴奋的能力之一是它们能够迭代推理复杂问题. 能否将同样的能力引入机器人, 通过在行动前推理给定任务, 让策略提高性能? 对于标准 VLAs 而言, 单纯使用"思路链"(CoT)风格提示的效果会差很多, 因为它们可用的训练示例相对简单. 此外, 对于(需要以感官观察和机器人状态为基础进行推理的)机器人策略而言, 关于子任务的纯语义推理(如常规 CoT 中常见的那样)是不够的. 为此, 我们为 VLAs 引入具身思维链推理 (ECoT), 在预测机器人动作之前, 我们训练 VLAs 对规划、子任务、运动和视觉基础特征(如物体边界框和末端执行器位置)执行多步推理. 我们设计了一个可扩展的流水线, 用于在大型机器人数据集上为 ECoT 生成合成训练数据. 我们证明, ECoT 在具有挑战性的泛化任务上, 将 OpenVLA (目前最强大的开源 VLA 策略)的绝对成功率提高了 28%, 无需任何额外的机器人训练数据. 此外, ECoT 使人类更容易解释策略的失败, 并使用自然语言纠正其行为.

# Introduction
<!-- Robotic policies trained end-to-end for outputting raw low-level actions in response to sensory observations provide a powerful and appealing learning-based approach to robotic control, obviating the need for complex sensing and control stacks, and processing complex observations into dexterous low-level controls [1, 2, 3]. However, this kind of "reactive" control is not necessarily the best choice in settings that demand broad generalization, such as novel scenes or interactions with unfamiliar objects. In such situations, a person might need to think more carefully – if they were asked to put fruit on one plate and vegetables on another, they might first try to figure out which objects are fruits and which are vegetables, rather than simply performing a learned skill from "muscle memory." In the same way, we would like our robotic policies to both perform well-practiced end-to-end control, and to "reason through" novel situations before grounding their commands into actions. Such reasoning might include identifying and locating task relevant objects, producing a plan to accomplish a task, and translating sub-tasks and observations into movements. -->
端到端训练的机器人策略根据感知观察输出原始的下层动作, 为机器人控制提供了一种强大且有吸引力的基于学习的方法, 无需复杂的感知和控制栈, 并将复杂的观察处理为灵巧的下层控制 [1, 2, 3]。然而, 这种"反应式"控制在需要广泛泛化的配置中, 不一定是最好的选择, 例如新场景或与不熟悉物体的交互。在这种情况下, 一个人可能需要更仔细地思考——如果要求他们把水果放在一个盘子里, 把蔬菜放在另一个盘子里, 他们可能首先试图弄清楚哪些物体是水果, 哪些是蔬菜, 而不是简单地根据"肌肉记忆"执行所学的技能。同样, 我们希望我们的机器人策略既能执行经过良好练习的端到端控制, 又能在将命令付诸行动之前, "推理"新情况。这种推理可能包括识别和定位与任务相关的对象, 制定一个计划以完成任务, 以及将子任务和观察转化为运动。

Vision-language-action models (VLAs) – pre-trained vision-language models (VLMs) fine-tuned to produce robot actions – have gained popularity as an approach for leveraging the diversity of Internet data captured within large foundation models [4] in a simple and scalable policy learning recipe. Despite achieving state-of-the-art performance across a wide range of tasks and robot embodiments [5, 6, 7], VLAs typically learn a direct mapping from observations to actions without any intermediate reasoning. However, there have been many recent works exploring how language models (which serve as the backbone of VLAs) can be prompted to textually "think step-by-step" about a given task. Such chain-of-thought reasoning (CoT) [8] significantly improves their performance on complex reasoning tasks and is now de-facto a standard practice in language modeling [9].

We thus hypothesize that we can similarly boost VLA performance by training them to **textually reason** about their plan, environment, and motions, thereby allowing them to produce more accurate and robust robot actions. However, simply applying the CoT techniques from language modeling to the robotics domain faces several challenges. For one, current VLAs build on relatively small, open-source VLMs that cannot match closed models in their ability to perform meaningful reasoning when simply prompted to think step-by-step [8]. Additionally, the most common CoT reasoning in language models, breaking tasks into sub-tasks, albeit helpful, is insufficient for reasoning about robotic tasks. The VLA policy needs to ground its plans and reasoning in its *observations* of the environment and robot state. Only then can the reasoning direct the agent’s attention toward fine-grained spatial or semantic perceptual features that are key for solving robot manipulation tasks. Put simply, we need VLAs to not only "think carefully", but also "look carefully."

To this end, we introduce **Embodied Chain-of-Thought Reasoning** (**ECoT**) for VLA policies. In contrast to prior VLAs, embodied chain-of-thought policies perform multiple steps of textual reasoning before predicting the next robot action (see Fig. 1, right). In contrast to existing CoT reasoning approaches for language models, they interleave semantic-level reasoning about sub-tasks with "embodied" reasoning tasks that require the policy to pay attention to its multi-modal inputs, from predicting bounding boxes of objects in the scene to reasoning about low-level movement primitives that need to be executed based on the current robot state. To enable the relatively weak LLM backbones of open-source VLAs to perform such reasoning effectively, we design a scalable pipeline for synthetically generating embodied CoT training data for large robot datasets. Concretely, we use powerful pre-trained open-vocabulary object detectors and large language models to generate the reasoning supervision for our policies.

Our experiments show that by training state-of-the-art VLAs to perform multiple steps of reasoning before action prediction, we can substantially boost their ability to perform challenging generalization tasks. Our approach increases the absolute success rate of OpenVLA [7], the current best-performing open-source VLA policy, by 28% across a suite of robot manipulation tasks that involve generalization to new objects, scenes, viewpoints, and instructions without any additional robot training data. Beyond raw performance improvements, our experiments show that training VLAs with embodied CoT makes policy failures more interpretable and allows humans to easily correct policy behavior by modifying faulty reasoning chains via natural language feedback.

In [5]:
from pypdf import PdfReader

pdf_path = "/mnt/d/cfz/github/Robotic Control via Embodied Chain-of-Thought Reasoning.pdf"

reader = PdfReader(pdf_path)
number_of_pages = len(reader.pages)
page = reader.pages[1]
text = page.extract_text()
print(text)

Vision-language-action models (VLAs) – pre-trained vision-language models (VLMs) fine-tuned
to produce robot actions – have gained popularity as an approach for leveraging the diversity of
Internet data captured within large foundation models [ 4] in a simple and scalable policy learning
recipe. Despite achieving state-of-the-art performance across a wide range of tasks and robot
embodiments [ 5,6,7], VLAs typically learn a direct mapping from observations to actions without any
intermediate reasoning. However, there have been many recent works exploring how language models
(which serve as the backbone of VLAs) can be prompted to textually “think step-by-step” about a
given task. Such chain-of-thought reasoning (CoT) [ 8] significantly improves their performance on
complex reasoning tasks and is now de-facto a standard practice in language modeling [9].
Regular VLA Policy
User: Clean the sink.OpenVLA:[x, , Grip] = …ΔΔθΔ
Chain-of-Thought
User: Clean the sink.
CoT OpenVLA:[x, , Grip] = …